In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# %cd drive/MyDrive/IMDB

/content/drive/MyDrive/IMDB


In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import seaborn as sns
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from gensim.parsing.preprocessing import remove_stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers, optimizers, losses
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import math

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
detail_df = pd.read_csv('./data/datasets/movie/all_details.csv', sep=',', encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning:

Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.



In [5]:
detail_df.shape

(80698, 21)

In [6]:
detail_df.isna().sum()

movie id                    0
title                       0
series                  67933
release year            17392
certification           19788
duration                23609
average rating           5947
rating total             5947
popularity score        69995
popularity delta        70022
content                  2833
numOfUserReviews        10977
numOfCriticReviews      23234
metaScore               67381
star_url_list            1093
countries_of_origin      1422
official_sites          44917
languages                2864
filming_locations       32838
production_companies     6147
budget_list             61521
dtype: int64

In [7]:
def isnan(value):
    try:
        import math
        return math.isnan(float(value))
    except:
        return False

In [8]:
for i in tqdm(range(len(detail_df))):
  if isnan(detail_df.iloc[i]['certification']):
    continue
  if detail_df.iloc[i]['certification'][0].isnumeric():
    detail_df.at[i, 'release year'] = detail_df.iloc[i]['certification']
    detail_df.at[i, 'certification'] = detail_df.iloc[i]['duration']
    detail_df.at[i, 'duration'] = None

100%|██████████| 80698/80698 [00:20<00:00, 3998.60it/s]


In [9]:
detail_df

,movie id,title,series,release year,certification,duration,average rating,rating total,popularity score,popularity delta,...,numOfUserReviews,numOfCriticReviews,metaScore,star_url_list,countries_of_origin,official_sites,languages,filming_locations,production_companies,budget_list
0,tt0378947,Melinda and Melinda,NaN,2004,PG-13,1h 39m,6.4,32K,NaN,NaN,...,184,155,54.0,/name/nm0002071/?ref_=tt_ov_st|/name/nm0005416...,United States,Official site,English,"Filming locations|Central Park, Manhattan, New...",Production companies|Fox Searchlight Pictures|...,NaN
1,tt0306047,Scary Movie 3,NaN,2003,PG-13,1h 24m,5.5,147K,"1,525",151,...,384,159,49.0,/name/nm0267506/?ref_=tt_ov_st|/name/nm0000221...,United States|Canada,Official sites|Miramax (United States)|Officia...,English,"Filming locations|Washington, District of Colu...",Production companies|Dimension Films|Brad Grey...,"$48,000,000 (estimated)"
2,tt4183480,Forevermore,NaN,2014–2015,TV-PGTV-PG,None,8.4,280,NaN,NaN,...,NaN,NaN,NaN,/name/nm6628668/?ref_=tt_ov_st|/name/nm4715046...,Philippines,NaN,Filipino,NaN,ABS-CBN,NaN
3,tt10483610,Capitani,TV Series,2019–2022,TV-MA,5h 12m,6.6,4.5K,543,"2,342",...,85,12,NaN,/name/nm5020632/?ref_=tt_ov_st|/name/nm8090479...,Luxembourg,Coproducer's website,Luxembourgish,Filming locations|Luxembourg|,Production companies|Artémis Productions|Film ...,NaN
4,tt9253298,Shaun the Sheep 3D,TV Mini Series,2012,Not Rated,1m,8.1,28,NaN,NaN,...,NaN,NaN,NaN,/name/nm1017994/?ref_=tt_ov_st,United Kingdom,Official site,NaN,NaN,Production company|Aardman Animations|,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80693,tt8847626,Dhund,NaN,2018,NaN,NaN,7.5,9,NaN,NaN,...,NaN,NaN,NaN,/name/nm3192510/?ref_=tt_ov_st|/name/nm1282023...,India,Large Short Winners,Hindi,NaN,Production companies|Fundamental Pictures|Hous...,NaN
80694,tt0025330,Jimmy the Gent,NaN,1934,Passed,1h 7m,6.6,1.9K,NaN,NaN,...,30,12.0,NaN,/name/nm0000010/?ref_=tt_ov_st|/name/nm0000012...,United States,NaN,English,Warner Brothers Burbank Studios - 4000 Warner ...,Production company|Warner Bros.|,NaN
80695,tt0900387,Suite Française,NaN,2014,NaN,NaN,7.0,26K,"4,750",849,...,81,109.0,63.0,/name/nm0931329/?ref_=tt_ov_st|/name/nm0000218...,United Kingdom|France|Canada|Belgium|United St...,Official sites|Official Facebook|Official site...,English|German|French|Latin,"Filming locations|Marville, Meuse, Lorraine, F...",Production companies|Alliance|Qwerty Films|Sco...,"€15,000,000 (estimated)"
80696,tt9098872,The Rescue,NaN,2021,PG,1h 47m,8.3,13K,"3,201",557,...,93,54.0,84.0,/name/nm12943039/?ref_=tt_ov_st|/name/nm129430...,United Kingdom|United States,Official site,Thai|English,Filming locations|Thailand|,Production companies|National Geographic Docum...,NaN


In [10]:
detail_df['series'] = detail_df['series'].fillna('Others')

In [11]:
detail_df = detail_df.dropna(subset=['release year'])

In [12]:
detail_df = detail_df.dropna(subset=['title'])

In [13]:
detail_df = detail_df.dropna(subset=['star_url_list'])

In [14]:
detail_df = detail_df.dropna(subset=['languages'])

In [15]:
detail_df = detail_df.dropna(subset=['countries_of_origin'])

In [16]:
# detail_df = detail_df.dropna(subset=['official_sites'])

In [17]:
# detail_df = detail_df.dropna(subset=['production_companies'])

In [18]:
# detail_df = detail_df.dropna(subset=['budget_list'])

In [19]:
detail_df['certification'].value_counts()

R            11597
Not Rated     8617
PG-13         4082
TV-14         3418
PG            3411
             ...  
3h 36m           1
7h 20m           1
5h 48m           1
9h 30m           1
8h 40m           1
Name: certification, Length: 362, dtype: int64

In [20]:
detail_df = detail_df[detail_df['certification'].isin(detail_df['certification'].value_counts().index.tolist()[:14])]

In [21]:
detail_df = detail_df.dropna(subset=['content'])

In [22]:
detail_df = detail_df.dropna(subset=['average rating'])

In [23]:
detail_df = detail_df.dropna(subset=['rating total'])

In [24]:
detail_df['release year']

0             2004
1             2003
3        2019–2022
6             2006
8             1986
           ...    
80686         2016
80687         1962
80692         1990
80694         1934
80696         2021
Name: release year, Length: 44954, dtype: object

In [25]:
def release_year_preprocessing(x):
  x = str(x)
  if (x.isnumeric() == False):
    x = x.strip()
  if x[:-1] == '–':
    x = x[:-1]
  elif x.find('–') != -1:
    x = 'period'
  
  if (x == 'period'):
    return 0
  if (int(x) >= 2017):
    return 1
  if (int(x) >= 2012):
    return 2
  if (int (x) >= 2007):
    return 3
  if (int (x) >= 2002):
    return 4
  if (int (x) >= 1997):
    return 5
  if (int (x) >= 1992):
    return 6
  return 7

detail_df['year'] = detail_df['release year'].apply(release_year_preprocessing)


In [26]:
# def convert_duration_2_num(x):
#   nums = ''
#   for c in x:
#     if c != 'h' and c != 'm':
#       nums +=c
#   nums = nums.split(" ")
#   if len(nums) == 1:
#     return int(nums[0])
#   else:
#     return int(nums[0]) * 60 + int(nums[1])  
# detail_df['movie runtime'] = detail_df['duration'].apply(convert_duration_2_num)

In [27]:
def cal_num_of_countries(x):
  return len(x.split('|'))
detail_df['num_of_countries'] = detail_df['countries_of_origin'].apply(cal_num_of_countries)
num_of_countries_vector = np.array(detail_df['num_of_countries'])

def cal_num_of_languages(x):
  return len(x.split('|'))
detail_df['num_of_languages'] = detail_df['languages'].apply(cal_num_of_languages)


In [28]:
full_content = []
content_list = list(detail_df['content'])
title_list = list(detail_df['title'])
for i in range(len(content_list)):
  full_content.append(' '.join([title_list[i], content_list[i]]))

In [29]:
detail_df['content'] = full_content

In [30]:
detail_df['content'] 

0        Melinda and Melinda Two alternating stories, o...
1        Scary Movie 3 Cindy must investigate mysteriou...
3        Capitani In a Luxembourg village where everyon...
6        Eum-lan-seo-seng A noble, high-class scholar s...
8        A Man and a Woman: 20 Years Later Jean-Louis a...
                               ...                        
80686    Eliminators Thomas, widower and ex federal age...
80687    The Horizontal Lieutenant A luckless army inte...
80692    Time Barbarians A medieval warrior chases a ba...
80694    Jimmy the Gent An unpolished racketeer, whose ...
80696    The Rescue A chronicle of the enthralling, aga...
Name: content, Length: 44954, dtype: object

In [31]:
import re
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

#Removes Emojis
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

detail_df['content']=detail_df['content'].apply(lambda z: remove_punctuations(z))

detail_df['content']=detail_df['content'].apply(lambda z: remove_html(z))
detail_df['content']=detail_df['content'].apply(lambda z: remove_url(z))

In [32]:
detail_df['content']

0        Melinda and Melinda Two alternating stories on...
1        Scary Movie 3 Cindy must investigate mysteriou...
3        Capitani In a Luxembourg village where everyon...
6        Eumlanseoseng A noble highclass scholar starts...
8        A Man and a Woman 20 Years Later JeanLouis and...
                               ...                        
80686    Eliminators Thomas widower and ex federal agen...
80687    The Horizontal Lieutenant A luckless army inte...
80692    Time Barbarians A medieval warrior chases a ba...
80694    Jimmy the Gent An unpolished racketeer whose r...
80696    The Rescue A chronicle of the enthralling agai...
Name: content, Length: 44954, dtype: object

In [33]:
def remove_abb(data):
    data = re.sub(r"he's", "he is", data)
    data = re.sub(r"there's", "there is", data)
    data = re.sub(r"We're", "We are", data)
    data = re.sub(r"That's", "That is", data)
    data = re.sub(r"won't", "will not", data)
    data = re.sub(r"they're", "they are", data)
    data = re.sub(r"Can't", "Cannot", data)
    data = re.sub(r"wasn't", "was not", data)
    data = re.sub(r"don\x89Ûªt", "do not", data)
    data= re.sub(r"aren't", "are not", data)
    data = re.sub(r"isn't", "is not", data)
    data = re.sub(r"What's", "What is", data)
    data = re.sub(r"haven't", "have not", data)
    data = re.sub(r"hasn't", "has not", data)
    data = re.sub(r"There's", "There is", data)
    data = re.sub(r"He's", "He is", data)
    data = re.sub(r"It's", "It is", data)
    data = re.sub(r"You're", "You are", data)
    data = re.sub(r"I'M", "I am", data)
    data = re.sub(r"shouldn't", "should not", data)
    data = re.sub(r"wouldn't", "would not", data)
    data = re.sub(r"i'm", "I am", data)
    data = re.sub(r"I\x89Ûªm", "I am", data)
    data = re.sub(r"I'm", "I am", data)
    data = re.sub(r"Isn't", "is not", data)
    data = re.sub(r"Here's", "Here is", data)
    data = re.sub(r"you've", "you have", data)
    data = re.sub(r"you\x89Ûªve", "you have", data)
    data = re.sub(r"we're", "we are", data)
    data = re.sub(r"what's", "what is", data)
    data = re.sub(r"couldn't", "could not", data)
    data = re.sub(r"we've", "we have", data)
    data = re.sub(r"it\x89Ûªs", "it is", data)
    data = re.sub(r"doesn\x89Ûªt", "does not", data)
    data = re.sub(r"It\x89Ûªs", "It is", data)
    data = re.sub(r"Here\x89Ûªs", "Here is", data)
    data = re.sub(r"who's", "who is", data)
    data = re.sub(r"I\x89Ûªve", "I have", data)
    data = re.sub(r"y'all", "you all", data)
    data = re.sub(r"can\x89Ûªt", "cannot", data)
    data = re.sub(r"would've", "would have", data)
    data = re.sub(r"it'll", "it will", data)
    data = re.sub(r"we'll", "we will", data)
    data = re.sub(r"wouldn\x89Ûªt", "would not", data)
    data = re.sub(r"We've", "We have", data)
    data = re.sub(r"he'll", "he will", data)
    data = re.sub(r"Y'all", "You all", data)
    data = re.sub(r"Weren't", "Were not", data)
    data = re.sub(r"Didn't", "Did not", data)
    data = re.sub(r"they'll", "they will", data)
    data = re.sub(r"they'd", "they would", data)
    data = re.sub(r"DON'T", "DO NOT", data)
    data = re.sub(r"That\x89Ûªs", "That is", data)
    data = re.sub(r"they've", "they have", data)
    data = re.sub(r"i'd", "I would", data)
    data = re.sub(r"should've", "should have", data)
    data = re.sub(r"You\x89Ûªre", "You are", data)
    data = re.sub(r"where's", "where is", data)
    data = re.sub(r"Don\x89Ûªt", "Do not", data)
    data = re.sub(r"we'd", "we would", data)
    data = re.sub(r"i'll", "I will", data)
    data = re.sub(r"weren't", "were not", data)
    data = re.sub(r"They're", "They are", data)
    data = re.sub(r"Can\x89Ûªt", "Cannot", data)
    data = re.sub(r"you\x89Ûªll", "you will", data)
    data = re.sub(r"I\x89Ûªd", "I would", data)
    data = re.sub(r"let's", "let us", data)
    data = re.sub(r"it's", "it is", data)
    data = re.sub(r"can't", "cannot", data)
    data = re.sub(r"don't", "do not", data)
    data = re.sub(r"you're", "you are", data)
    data = re.sub(r"i've", "I have", data)
    data = re.sub(r"that's", "that is", data)
    data = re.sub(r"i'll", "I will", data)
    data = re.sub(r"doesn't", "does not",data)
    data = re.sub(r"i'd", "I would", data)
    data = re.sub(r"didn't", "did not", data)
    data = re.sub(r"ain't", "am not", data)
    data = re.sub(r"you'll", "you will", data)
    data = re.sub(r"I've", "I have", data)
    data = re.sub(r"Don't", "do not", data)
    data = re.sub(r"I'll", "I will", data)
    data = re.sub(r"I'd", "I would", data)
    data = re.sub(r"Let's", "Let us", data)
    data = re.sub(r"you'd", "You would", data)
    data = re.sub(r"It's", "It is", data)
    data = re.sub(r"Ain't", "am not", data)
    data = re.sub(r"Haven't", "Have not", data)
    data = re.sub(r"Could've", "Could have", data)
    data = re.sub(r"youve", "you have", data)  
    data = re.sub(r"donå«t", "do not", data)  
    return data

In [34]:
detail_df['content']=detail_df['content'].apply(lambda z: remove_abb(z))

In [35]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [36]:
from nltk.stem import WordNetLemmatizer


def lemma_traincorpus(data):
    lemmatizer=WordNetLemmatizer()
    out_data=""
    for words in data:
        out_data+= lemmatizer.lemmatize(words)
    return out_data

detail_df['content']=detail_df['content'].apply(lambda z: lemma_traincorpus(z))

from nltk.stem import *
def stem_traincorpus(data):
    stemmer = PorterStemmer()
    out_data=""
    for words in data:
        out_data+= stemmer.stem(words)
    return out_data

detail_df['content']=detail_df['content'].apply(lambda z: stem_traincorpus(z))
detail_df['content']

0        melinda and melinda two alternating stories on...
1        scary movie 3 cindy must investigate mysteriou...
3        capitani in a luxembourg village where everyon...
6        eumlanseoseng a noble highclass scholar starts...
8        a man and a woman 20 years later jeanlouis and...
                               ...                        
80686    eliminators thomas widower and ex federal agen...
80687    the horizontal lieutenant a luckless army inte...
80692    time barbarians a medieval warrior chases a ba...
80694    jimmy the gent an unpolished racketeer whose r...
80696    the rescue a chronicle of the enthralling agai...
Name: content, Length: 44954, dtype: object

In [37]:
from nltk.stem import WordNetLemmatizer


def lemma_traincorpus(data):
    lemmatizer=WordNetLemmatizer()
    out_data=""
    for words in data:
        out_data+= lemmatizer.lemmatize(words)
    return out_data

detail_df['content']=detail_df['content'].apply(lambda z: lemma_traincorpus(z))

In [38]:
from nltk.stem import *
def stem_traincorpus(data):
    stemmer = PorterStemmer()
    out_data=""
    for words in data:
        out_data+= stemmer.stem(words)
    return out_data

detail_df['content']=detail_df['content'].apply(lambda z: stem_traincorpus(z))
detail_df['content']

0        melinda and melinda two alternating stories on...
1        scary movie 3 cindy must investigate mysteriou...
3        capitani in a luxembourg village where everyon...
6        eumlanseoseng a noble highclass scholar starts...
8        a man and a woman 20 years later jeanlouis and...
                               ...                        
80686    eliminators thomas widower and ex federal agen...
80687    the horizontal lieutenant a luckless army inte...
80692    time barbarians a medieval warrior chases a ba...
80694    jimmy the gent an unpolished racketeer whose r...
80696    the rescue a chronicle of the enthralling agai...
Name: content, Length: 44954, dtype: object

In [39]:
detail_df['content']=detail_df['content'].apply(lambda z: remove_stopwords(z))

In [40]:
signs = set()
for x in detail_df['rating total']:
  for c in x:
    if c.isnumeric():
      continue
    signs.add(c)
signs

{'.', 'K', 'M'}

In [41]:
def convert_rating_total_2_num(x):
  if 'K' in x:
    return int(float(x[:-1]) * 1000)
  if 'M' in x:
    return int(float(x[:-1]) * 100000)
  return x

In [42]:
detail_df['rating total'] = detail_df['rating total'].apply(convert_rating_total_2_num)

In [43]:
rating = []
rating_total_list = list(detail_df['rating total'])
average_rating_list = list(detail_df['average rating'])
for i in range(len(detail_df['rating total'])):
  rating.append(float(rating_total_list[i]) * float(average_rating_list[i]))

In [44]:
detail_df['rating sum'] = rating
def scale_rating_sum(x):
  return np.log(x)
detail_df['rating sum'] = detail_df['rating sum'].apply(scale_rating_sum)

In [45]:
detail_df.to_csv('./data/datasets/movie/cleaned_preprocessed_1_detail.csv', sep=',', encoding='utf-8', index=False)